In [1]:
import os
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [14]:
#Creating function to return dataframe with tweets
#Filters replies and links
def get_tweets(username,n=1000):
    output = []
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:'+username+' -filter:replies -filter:links').get_items()):
        if i>n:
            break
        output.append([tweet.date, tweet.id, tweet.content, tweet.user.username])
    return pd.DataFrame(output, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])

In [18]:
#Testing
get_tweets('c9mang0',n=20)

,Datetime,Tweet Id,Text,Username
0,2021-02-18 18:46:55+00:00,1362473690395385856,Eagles are such a shit show ROFL,C9Mang0
1,2021-02-18 10:54:06+00:00,1362354698733252610,Not getting diamond is driving me insane \n\nM...,C9Mang0
2,2021-02-17 22:18:46+00:00,1362164614088118273,Nvm \n\nNo online for Mario golf ? \n\nNvm we ...,C9Mang0
3,2021-02-17 22:16:55+00:00,1362164149145378819,Fucking bitch shit fucker \n\nI said if they m...,C9Mang0
4,2021-02-17 22:03:40+00:00,1362160812366778368,How old is the new character,C9Mang0
5,2021-02-15 03:01:04+00:00,1361148494954729473,Might stream later tonight if I don't pass out...,C9Mang0
6,2021-02-13 22:15:11+00:00,1360714158979784704,Me bannana,C9Mang0
7,2021-02-12 11:38:50+00:00,1360191629479399427,Rocket league is crack,C9Mang0
8,2021-02-10 00:05:10+00:00,1359292287503589382,Last minute entered the clg volleyball thing\n...,C9Mang0
9,2021-02-09 00:27:43+00:00,1358935573382090754,I'll be running a little late but ill be on ev...,C9Mang0
